# Projek Akhir Data Engineering

#### Anggota Kelompok :
1. Nurfatima Selle - 175150201111058
2. Jeowandha Ria Wiyani - 175150207111029
3. Adinda Chilliya Basuki -175150207111040

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import *

if __name__ == '__main__':
    scSpark = SparkSession \
        .builder \
        .appName("pipeline spark") \
        .config("spark.driver.extraClassPath", 
                "/Users/WINDOWS 10/spark/spark-2.4.7-bin-hadoop2.7/jars/mysql-connector-java-8.0.22.jar") \
        .getOrCreate()

#### Mengambil Datasource

In [2]:
#Extract
dataset = 'C:/Users/WINDOWS 10/kc_house_data.csv' 
datasetDf = scSpark.read.csv(dataset, header=True, sep=",").cache()

datasetDf.show(5)
datasetDf.printSchema()

+----------+---------------+------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|        id|           date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|
+----------+---------------+------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|7129300520|20141013T000000|221900|       3|        1|       1180|    5650|     1|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|47.5112|-122.257|         1340|      5650|
|6414100192|20141209T000000|538000|       3|     2.25|       2570|    7242|     2|         0|   0|        3|    7|      2170|          400|    1951|    

#### Menyesuaikan Schema
Mengubah beberapa tipe data dari kolom

In [3]:
datasetDf = datasetDf.withColumn("date", datasetDf["date"].cast(DateType()))
datasetDf = datasetDf.withColumn("price", datasetDf["price"].cast(FloatType()))
datasetDf = datasetDf.withColumn("bedrooms", datasetDf["bedrooms"].cast(FloatType()))
datasetDf = datasetDf.withColumn("bathrooms", datasetDf["bathrooms"].cast(FloatType()))
datasetDf = datasetDf.withColumn("sqft_living", datasetDf["sqft_living"].cast(FloatType()))
datasetDf = datasetDf.withColumn("sqft_lot", datasetDf["sqft_lot"].cast(FloatType()))
datasetDf = datasetDf.withColumn("floors", datasetDf["floors"].cast(FloatType()))
datasetDf = datasetDf.withColumn("waterfront", datasetDf["waterfront"].cast(FloatType()))
datasetDf = datasetDf.withColumn("view", datasetDf["view"].cast(FloatType()))
datasetDf = datasetDf.withColumn("condition", datasetDf["condition"].cast(FloatType()))
datasetDf = datasetDf.withColumn("grade", datasetDf["grade"].cast(FloatType()))
datasetDf = datasetDf.withColumn("sqft_above", datasetDf["sqft_above"].cast(FloatType()))
datasetDf = datasetDf.withColumn("yr_built", datasetDf["yr_built"].cast(IntegerType()))
datasetDf = datasetDf.withColumn("yr_renovated", datasetDf["yr_renovated"].cast(IntegerType()))
datasetDf = datasetDf.withColumn("sqft_living15", datasetDf["sqft_living15"].cast(FloatType()))
datasetDf = datasetDf.withColumn("sqft_lot15", datasetDf["sqft_lot15"].cast(FloatType()))

datasetDf.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- price: float (nullable = true)
 |-- bedrooms: float (nullable = true)
 |-- bathrooms: float (nullable = true)
 |-- sqft_living: float (nullable = true)
 |-- sqft_lot: float (nullable = true)
 |-- floors: float (nullable = true)
 |-- waterfront: float (nullable = true)
 |-- view: float (nullable = true)
 |-- condition: float (nullable = true)
 |-- grade: float (nullable = true)
 |-- sqft_above: float (nullable = true)
 |-- sqft_basement: string (nullable = true)
 |-- yr_built: integer (nullable = true)
 |-- yr_renovated: integer (nullable = true)
 |-- zipcode: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- sqft_living15: float (nullable = true)
 |-- sqft_lot15: float (nullable = true)



#### Preprocess

In [4]:
cleanDatasetDf = datasetDf.drop('date','waterfront','view','lat','sqft_living15','sqft_lot15','long', 'zipcode')
cleanDatasetDf.show(5)

+----------+--------+--------+---------+-----------+--------+------+---------+-----+----------+-------------+--------+------------+
|        id|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|
+----------+--------+--------+---------+-----------+--------+------+---------+-----+----------+-------------+--------+------------+
|7129300520|221900.0|     3.0|      1.0|     1180.0|  5650.0|   1.0|      3.0|  7.0|    1180.0|            0|    1955|           0|
|6414100192|538000.0|     3.0|     2.25|     2570.0|  7242.0|   2.0|      3.0|  7.0|    2170.0|          400|    1951|        1991|
|5631500400|180000.0|     2.0|      1.0|      770.0| 10000.0|   1.0|      3.0|  6.0|     770.0|            0|    1933|           0|
|2487200875|604000.0|     4.0|      3.0|     1960.0|  5000.0|   1.0|      5.0|  7.0|    1050.0|          910|    1965|           0|
|1954400510|510000.0|     3.0|      2.0|     1680.0|  8080.0|   1.0|      3.

#### Memasukkan ke Database

In [5]:
cleanDatasetDf.write.format('jdbc').options(
    url='jdbc:mysql://localhost/dataengineering',
    driver='com.mysql.cj.jdbc.Driver',
    dbtable='house_price',
    user='root').mode('append').save()

#### Ambil Subset Data dari Database

In [6]:
# !pip install mysql-connector-python

In [7]:
import mysql.connector
import pandas as pd

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  database="dataengineering"
)

cursor = mydb.cursor()
cursor.execute("SELECT price, bedrooms, bathrooms, sqft_living, sqft_lot, grade FROM house_price")

hasilSubset = cursor.fetchall()
subsetDf = scSpark.createDataFrame(hasilSubset).toDF("price", "bedrooms", "bathrooms", "sqft_living", "sqft_lot", "grade")
subsetPandasDf = subsetDf.toPandas()

#### Proses EDA

In [8]:
subsetPandasDf.head(5)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,grade
0,221900.0,3.0,1.00,1180.0,5650.0,7.0
1,538000.0,3.0,2.25,2570.0,7242.0,7.0
2,180000.0,2.0,1.00,770.0,10000.0,6.0
3,604000.0,4.0,3.00,1960.0,5000.0,7.0
4,510000.0,3.0,2.00,1680.0,8080.0,8.0


In [9]:
subsetPandasDf.shape

(216130, 6)

In [10]:
subsetPandasDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216130 entries, 0 to 216129
Data columns (total 6 columns):
price          216130 non-null float64
bedrooms       216130 non-null float64
bathrooms      216130 non-null float64
sqft_living    216130 non-null float64
sqft_lot       216130 non-null float64
grade          216130 non-null float64
dtypes: float64(6)
memory usage: 9.9 MB


In [11]:
subsetPandasDf.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,grade
count,2.161300e+05,216130.000000,216130.000000,216130.000000,2.161300e+05,216130.000000
mean,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,7.656873
std,3.671196e+05,0.930042,0.770147,918.421774,4.141965e+04,1.175434
min,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000
25%,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,7.000000
50%,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,7.000000
75%,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,8.000000
max,7.700000e+06,33.000000,8.000000,13540.000000,1.651360e+06,13.000000


In [12]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, axes = plt.subplots(figsize=(20,10))
corrResult = sns.heatmap(subsetPandasDf, cmap='coolwarm', annot=True, annot_kws={'size':10}, fmt=".7f", ax=axes)

#### Proses Machine Learning

In [13]:
import numpy as np
import multiprocessing as mp
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from multiprocessing import Pool
import time

In [14]:
x = subsetPandasDf.drop(['price'], axis=1)
y = subsetPandasDf['price'].values.reshape(-1,1)
print(x, y)

        bedrooms  bathrooms  sqft_living  sqft_lot  grade
0            3.0       1.00       1180.0    5650.0    7.0
1            3.0       2.25       2570.0    7242.0    7.0
2            2.0       1.00        770.0   10000.0    6.0
3            4.0       3.00       1960.0    5000.0    7.0
4            3.0       2.00       1680.0    8080.0    8.0
...          ...        ...          ...       ...    ...
216125       3.0       2.50       1530.0    1131.0    8.0
216126       4.0       2.50       2310.0    5813.0    8.0
216127       2.0       0.75       1020.0    1350.0    7.0
216128       3.0       2.50       1600.0    2388.0    8.0
216129       2.0       0.75       1020.0    1076.0    7.0

[216130 rows x 5 columns] [[221900.]
 [538000.]
 [180000.]
 ...
 [402101.]
 [400000.]
 [325000.]]


In [15]:
def regresi(x_train, y_train, x_test, y_test):
  # Mengaktifkan/memanggil/membuat fungsi regresi linear
  lin_reg = LinearRegression()
  model = lin_reg.fit(x_train, y_train)
  # make predictions
  y_pred = model.predict(x_test)
  # menghitung akurasi
  akurasi =  model.score(x_test, y_test)
  hasil=pd.DataFrame({'Harga Asli':y_test.flatten(),'Harga Prediksi':y_pred.flatten()})
  print(hasil.head(10))
  print("\nAkurasi hasil prediksi: ", str(akurasi))
  return akurasi, hasil

In [16]:
%%time
with Pool(5) as p:
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
  tasks = [(x_train, x_test, y_train, y_test)]
  result = p.map(regresi, tasks)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
x_train2, x_test2, y_train2, y_test2 = train_test_split(x, y, test_size=0.3)

In [18]:
%%time
regresi(x_train, y_train, x_test, y_test)

   Harga Asli  Harga Prediksi
0    261590.0   165582.058607
1    235000.0   280128.587011
2    760000.0   977248.306870
3    242000.0   400736.188823
4    785000.0   501258.628229
5    275500.0   154678.171603
6    734000.0   766223.222435
7    454000.0   371528.110536
8    400000.0   421256.314053
9    251000.0   144417.235467

Akurasi hasil prediksi:  0.5492102339023104
Wall time: 1.22 s


(0.5492102339023104,        Harga Asli  Harga Prediksi
 0        261590.0   165582.058607
 1        235000.0   280128.587011
 2        760000.0   977248.306870
 3        242000.0   400736.188823
 4        785000.0   501258.628229
 ...           ...             ...
 43221    850000.0   713390.316016
 43222    720000.0   872083.890588
 43223    218000.0   265884.162462
 43224    630000.0   447845.893239
 43225    530200.0   494139.831155
 
 [43226 rows x 2 columns])

In [19]:
%%time
regresi(x_train2, y_train2, x_test2, y_test2)

   Harga Asli  Harga Prediksi
0    235000.0    3.323803e+05
1    438000.0    3.452599e+05
2    295000.0    8.590181e+04
3    583000.0    1.031542e+06
4    309500.0    3.347410e+05
5    540000.0    3.900576e+05
6    960000.0    6.585113e+05
7    870000.0    1.030813e+06
8    651000.0    4.661497e+05
9   1378000.0    9.351744e+05

Akurasi hasil prediksi:  0.5437450302938245
Wall time: 237 ms


(0.5437450302938245,        Harga Asli  Harga Prediksi
 0        235000.0    3.323803e+05
 1        438000.0    3.452599e+05
 2        295000.0    8.590181e+04
 3        583000.0    1.031542e+06
 4        309500.0    3.347410e+05
 ...           ...             ...
 64834   1047500.0    8.784279e+05
 64835    750000.0    5.840924e+05
 64836    442500.0    3.933319e+05
 64837   1185000.0    1.236508e+06
 64838    270000.0    2.978707e+05
 
 [64839 rows x 2 columns])